In [1]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import sys
from geopy.distance import vincenty

Магический стриппер! (ﾉ◕ヮ◕)ﾉ*:･ﾟ✧

In [2]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

Еще одна магическая штука, которую я нашел на просторах гугла - она как-то работает! Это пригодится нам потом, когда мы будем сортировать массив строк в числовом порядке.

In [3]:
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

Поиграв немного с ВАО, я понял, что можно сделать оптимальное разбиение по цене (например, 6.5М рублей), и тогда в каждом запросе будет не более 840 квартир.

In [4]:
urls = ['http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D={}&engine_version=2&maxprice=6500000&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D={}&engine_version=2&minprice=6500001&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1']

In [20]:
start_time = time.time()
links = []
# 56, ..., 71 - номера районов, соответствующих ВАО
for dist in range(56, 72):
    start_time_dist = time.time()
    last_len_links = len(links)
    print('DISTRICT {}'.format(dist))
    for url in urls:
        # Если по запросу выдается, скажем, 16 страниц, то если мы перескочим на 17, то нас перенаправит
        # на первую. Переменная is_looped отвечает за то, чтобы мы не зациклились
        is_looped = 0
        cnt = 0
        page = 1
        while is_looped == 0:
            # тут мы просто собираем информацию обо всех объявлениях на странице
            # (при помощи пристального вглядывания в html-код и профессионального кодинга (͡° ͜ʖ ͡°))
            page_url =  url.format(dist, page)
            search_page = BeautifulSoup(requests.get(page_url).content, 'lxml')
            pages = search_page.findAll('div', attrs = {'class':"pager_pages"})
            pages = re.split('<span>|</span>', str(pages))
            # количество предложений - эта переменная нужна потому, что схема антизацикливания
            # не работает, если все предложения на одной странице (т.е. их не больше 28)
            offers_number = search_page.findAll('div', attrs = {'class':"serp-above__count"})
            offers_number = int(re.split('<strong>|</strong>', str(offers_number))[1])
            # условие зацикливания
            if cnt != 0 and ('1' in pages or offers_number <= 28):
                print('Total pages: ', cnt)
                print('Total urls: ', len(links) - last_len_links)
                print('Unique urls: ', len(list(set(links))) - last_len_links)
                is_looped = 1
            # если не зациклились, то достаем все ссылки и переходим к следующей странице
            else:
                flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
                flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))
                cnt0 = 0
                for link in flat_urls:
                    if link.isdigit():
                        links.append(link)
                        cnt0 += 1
                print('Number of flat urls on the page {}: '.format(page), cnt0)
                page += 1
                cnt += 1
    # нам нужны только уникальные ссылки - как видно из вывода, некоторые объявления повторяются
    links = list(set(links))
    print('Total unique flat urls for district {}: '.format(dist), len(links) - last_len_links)
    print('Time spent on district {}: {} minutes'.format(dist, (time.time() - start_time_dist) / 60))
print('Total time spent: {} munutes'.format((time.time() - start_time) / 60))
print('Total unique flat urls: {}'.format(len(links)))

DISTRICT 56
Number of flat urls on the page 1:  26
Number of flat urls on the page 2:  26
Number of flat urls on the page 3:  26
Number of flat urls on the page 4:  26
Number of flat urls on the page 5:  26
Number of flat urls on the page 6:  26
Number of flat urls on the page 7:  18
Total pages:  7
Total urls:  174
Unique urls:  168
Number of flat urls on the page 1:  28
Number of flat urls on the page 2:  28
Number of flat urls on the page 3:  28
Number of flat urls on the page 4:  28
Number of flat urls on the page 5:  28
Number of flat urls on the page 6:  28
Number of flat urls on the page 7:  28
Number of flat urls on the page 8:  28
Number of flat urls on the page 9:  28
Number of flat urls on the page 10:  28
Number of flat urls on the page 11:  28
Number of flat urls on the page 12:  28
Number of flat urls on the page 13:  28
Number of flat urls on the page 14:  28
Number of flat urls on the page 15:  28
Number of flat urls on the page 16:  28
Number of flat urls on the page 1

Ура, мы получили массив номеров ссылок! Отсортируем его, вспомнив про магическую функцию сравнения в самом начале.

In [21]:
links = sorted(links, key=numericalSort)

Теперь займемся функциями, с помощью которых мы сможем доставать интересующую нас информацию со страничек.
Начнем с

> **N – номер квартиры по порядку**

In [5]:
def getN(link):
    return int(link)

(это было слишком просто)
> **Rooms – число комнат**

In [6]:
def getRooms(flat_page):
    rooms = flat_page.find('div', attrs={'class':"object_descr_title"})
    rooms = re.split(' |-', str(rooms))
    rooms = ''.join([i for i in rooms if i.isdigit()])
    # если квартира многокомнатная (6+), то количества комнат в том месте, где мы ищем, не стоит, и
    # у нас вылетает ValueError. Искать это самое число непросто - на разных подобных страничках
    # оно может стоять в разных местах, может быть в тексте, а может вообще не существовать. Поэтому
    # я решил не запариваться и выводить в подобных случаях 6, благо таких случаев на весь ВАО 11
    try:
        return int(rooms)
    except ValueError:
        return 6

> **Price – цена**

В цене на сайте стоят пробельчики, но мы их убираем

In [7]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = ''.join([i for i in price if i.isdigit()][-3:])
    return int(price)

> **Totsp – общая площадь квартиры, кв. м.**

In [8]:
def getTotsp(flat_page):
    totsp = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    totsp = html_stripper(totsp).replace('\n', '')
    totsp = re.split('Общая площадь:|\ха0м2', totsp)
    totsp = re.split('\xa0м2', totsp[1])
    # бывает, что некоторых данных на странице нет, как, например, общей площади квартиры.
    # В таких случаях при ValueError мы будем выводить -999999 и хитро с этим справляться в самом конце
    try:
        return float(totsp[0].replace(',', '.'))
    except ValueError:
        return -999999

> **Livesp – жилая площадь квартиры, кв. м.**

(аналогично)

In [9]:
def getLivesp(flat_page):
    livesp = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    livesp = html_stripper(livesp).replace('\n', '')
    livesp = re.split('Жилая площадь:|\ха0м2', livesp)
    livesp = re.split('\xa0м2', livesp[1])
    try:
        return float(livesp[0].replace(',', '.'))
    except ValueError:
        return -999999

> **Kitsp – площадь кухни, кв. м.**

(аналогично)

In [10]:
def getKitsp(flat_page):
    kitsp = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    kitsp = html_stripper(kitsp).replace('\n', '')
    kitsp = re.split('Площадь кухни:|\ха0м2', kitsp)
    kitsp = re.split('\xa0м2', kitsp[1])
    try:
        return float(kitsp[0].replace(',', '.'))
    except ValueError:
        return -999999

> **Dist – расстояние от центра в км. (можно по Пифагору)**

In [11]:
def getDist(flat_page):
    # вытаскиваем координаты из карты
    coords = flat_page.find('div', attrs={'class':"map_info_button_extend"}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    flat_coords = (float(coords_list[0]), float(coords_list[1]))
    # загугленные координаты центра Москвы
    mos_cent_coords = (55.753559, 37.609218)
    try:
        # пользуемся библиотекой geopy
        return vincenty(flat_coords, mos_cent_coords).kilometers
    except ValueError:
        return -999999

> **Metrdist – расстояние до метро в минутах**

In [12]:
def getMetrdist(flat_page):
    metrdist = flat_page.find('span', attrs={'class': "object_item_metro_comment"})
    metrdist = html_stripper(metrdist).replace(' ', '')
    metrdist = metrdist.split('\n')
    dists = [int(i) for i in metrdist if i.isdigit()]
    try:
        return int(min(dists))
    except ValueError:
        return -999999

> **Walk – 1 – если пешком от метро, 0 – если на транспорте**

In [13]:
def getWalk(flat_page):
    getwalk = flat_page.find('span', attrs={'class': "object_item_metro_comment"})
    getwalk = html_stripper(getwalk).replace(' ', '')
    if 'пешк' in getwalk:
        return 1
    else:
        return 0

> **Brick – 1 – кирпичный/монолит/жб, 0 – другой**

Вообще-то информация о стенах дома может быть в двух разных местах страницы, поэтому я проверяю оба

In [14]:
def getBrick(flat_page):
    getbrick = flat_page.find('table', attrs={'class': "bti__data"})
    getbrick = html_stripper(getbrick).replace('\n', '')
    getbrick = re.split('Материалы стен|Этажность', getbrick)
    if len(getbrick) != 1:
        if 'кирп' in getbrick[1] or 'мон' in getbrick[1] or 'жб' in getbrick[1]\
                               or'ж/б' in getbrick[1] or 'железобетон' in getbrick[1]:
            return 1
        else:
            return 0
    else:
        getbrick = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
        getbrick = html_stripper(getbrick).replace('\n', '')
        getbrick = re.split('Тип дома:|Тип продажи:', getbrick)
        try:
            if 'кирп' in getbrick[1] or 'мон' in getbrick[1] or 'жб' in getbrick[1]\
                               or'ж/б' in getbrick[1] or 'железобетон' in getbrick[1]:
                return 1
            else:
                return 0
        except ValueError:
            return -999999

> **Tel – 1 – если есть, 0 – нет**

In [15]:
def getTel(flat_page):
    gettel = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    gettel = html_stripper(gettel).replace('\n', '')
    gettel = re.split('Телефон:|Вид из окна:', gettel)
    if len(gettel) != 3 or 'не' in gettel[1] or '–' in gettel[1]:
        return 0
    else:
        return 1

> **Bal – 1 – есть балкон/лоджия, 0 – нет**

In [16]:
def getBal(flat_page):
    getbal = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    getbal = html_stripper(getbal).replace('\n', '')
    getbal = re.split('Балкон:|Лифт:', getbal)
    if len(getbal) != 3 or 'не' in getbal[1] or '–' in getbal[1]:
        return 0
    else:
        return 1

> **Floor – номер этажа, на котором расположена квартира**

In [17]:
def getFloor(flat_page):
    getfloor = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    getfloor = html_stripper(getfloor).replace('\n', '')
    getfloor = re.split('Этаж:|\xa0', getfloor)
    try:
        return int(getfloor[1])
    except ValueError:
        return -999999

> **Nfloors – всего этажей в доме**

In [18]:
def getNfloors(flat_page):
    getnfloors = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    getnfloors = html_stripper(getnfloors).replace('\n', '')
    getnfloors = re.split('/\xa0|Тип дома:', getnfloors)
    try:
        return int(getnfloors[1])
    except ValueError:
        return -999999

> **New – 1 – первичный рынок, 0 - вторичка**

In [19]:
def getNew(flat_page):
    getnew = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    getnew = html_stripper(getnew).replace('\n', '')
    getnew = re.split('Тип дома:|Высота потолков:', getnew)
    if 'втор' in getnew[1]:
        return 0
    else:
        return 1

Ссылки есть, функции есть, мы готовы гулять по страничкам!

In [22]:
start_time = time.time()
columns = ['N', 'Rooms', 'Price', 'Totsp', 'Livesp', 'Kitsp', 'Dist', 'Metrdist', 'Walk', 'Brick',
          'Tel', 'Bal', 'Floor', 'Nfloors', 'New']
df_final = pd.DataFrame(columns=columns)
for link in links:
    flat_page_url = 'http://www.cian.ru/sale/flat/' + link + '/'
    print('{}/{} Processing link {} ...'.format(links.index(link) + 1, len(links), flat_page_url))
    flat_page = BeautifulSoup(requests.get(flat_page_url).content, 'lxml')
    df_final.loc[len(df_final)] = [getN(link), getRooms(flat_page), getPrice(flat_page),
                         getTotsp(flat_page), getLivesp(flat_page), getKitsp(flat_page),
                         getDist(flat_page), getMetrdist(flat_page), getWalk(flat_page),
                         getBrick(flat_page), getTel(flat_page), getBal(flat_page),
                         getFloor(flat_page), getNfloors(flat_page), getNew(flat_page)]
print('Time spent: {} minutes'.format((time.time() - start_time) / 60))

1/7007 Processing link http://www.cian.ru/sale/flat/1338305/ ...
2/7007 Processing link http://www.cian.ru/sale/flat/1535194/ ...
3/7007 Processing link http://www.cian.ru/sale/flat/1711530/ ...
4/7007 Processing link http://www.cian.ru/sale/flat/1838610/ ...
5/7007 Processing link http://www.cian.ru/sale/flat/1986554/ ...
6/7007 Processing link http://www.cian.ru/sale/flat/2222278/ ...
7/7007 Processing link http://www.cian.ru/sale/flat/2292702/ ...
8/7007 Processing link http://www.cian.ru/sale/flat/2343853/ ...
9/7007 Processing link http://www.cian.ru/sale/flat/2464298/ ...
10/7007 Processing link http://www.cian.ru/sale/flat/2515421/ ...
11/7007 Processing link http://www.cian.ru/sale/flat/2547927/ ...
12/7007 Processing link http://www.cian.ru/sale/flat/2621950/ ...
13/7007 Processing link http://www.cian.ru/sale/flat/2684980/ ...
14/7007 Processing link http://www.cian.ru/sale/flat/2721689/ ...
15/7007 Processing link http://www.cian.ru/sale/flat/2721893/ ...
16/7007 Processing 

ConnectionError: HTTPConnectionPool(host='www.cian.ru', port=80): Max retries exceeded with url: /sale/flat/147429713/ (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x0799D0F0>: Failed to establish a new connection: [WinError 10051] Сделана попытка выполнить операцию на сокете при отключенной сети',))

Не знаю, что это - похоже на шалости интернета. Но давайте хотя бы посмотрим на то, что получилось на данный момент.

In [23]:
df_final.head(20)

,N,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Tel,Bal,Floor,Nfloors,New
0,1338305.0,2.0,9100000.0,56.4,35.1,7.0,8.942096,15.0,1.0,1.0,1.0,0.0,2.0,5.0,0.0
1,1535194.0,2.0,52000000.0,92.0,42.0,18.0,5.830754,1.0,1.0,1.0,0.0,0.0,2.0,30.0,0.0
2,1711530.0,2.0,6600000.0,44.5,28.0,6.0,13.741006,13.0,1.0,0.0,1.0,0.0,7.0,9.0,0.0
3,1838610.0,3.0,6600000.0,52.3,36.1,6.0,16.486721,10.0,0.0,1.0,1.0,0.0,1.0,5.0,0.0
4,1986554.0,3.0,13300000.0,80.4,54.8,8.6,7.735389,10.0,0.0,1.0,1.0,1.0,7.0,8.0,0.0
5,2222278.0,2.0,7900000.0,56.0,30.0,6.0,7.068869,5.0,1.0,1.0,0.0,0.0,1.0,8.0,0.0
6,2292702.0,3.0,15000000.0,73.5,45.0,10.0,13.470486,9.0,1.0,0.0,0.0,1.0,13.0,17.0,0.0
7,2343853.0,3.0,9700000.0,70.4,48.3,7.0,6.834289,12.0,1.0,1.0,1.0,1.0,5.0,6.0,0.0
8,2464298.0,1.0,14500000.0,58.0,25.0,15.0,11.137224,12.0,1.0,1.0,0.0,1.0,8.0,18.0,0.0
9,2515421.0,5.0,20000000.0,130.0,90.0,8.0,11.272828,11.0,1.0,1.0,1.0,0.0,1.0,5.0,0.0


In [24]:
df_final.tail()

,N,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Tel,Bal,Floor,Nfloors,New
1336,147427761.0,1.0,12300000.0,49.0,20.0,14.0,8.232253,-999999.0,0.0,1.0,0.0,1.0,3.0,14.0,1.0
1337,147428623.0,3.0,9350000.0,64.0,37.0,7.0,8.220883,15.0,1.0,1.0,0.0,0.0,5.0,5.0,0.0
1338,147428635.0,2.0,23800000.0,77.0,41.5,11.5,10.812791,7.0,1.0,1.0,1.0,1.0,4.0,18.0,0.0
1339,147429466.0,4.0,12500000.0,97.8,68.9,9.0,9.359785,8.0,1.0,1.0,1.0,1.0,4.0,5.0,0.0
1340,147429513.0,2.0,6250000.0,44.4,30.0,6.0,13.776372,3.0,0.0,0.0,0.0,0.0,5.0,9.0,0.0


Далее становится понятно, почему для пропущенного значения использовалось -999999, а не nan: мы переводим большинство столбцов в int (зачем, например, номеру ссылки тип 'float'?..)

In [25]:
df_final[['N', 'Rooms', 'Price', 'Metrdist', 'Walk', 'Brick',
          'Tel', 'Bal', 'Floor', 'Nfloors', 'New']] = \
df_final[['N', 'Rooms', 'Price', 'Metrdist', 'Walk', 'Brick',
          'Tel', 'Bal', 'Floor', 'Nfloors', 'New']].astype(int)
df_final.head(20)

,N,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Tel,Bal,Floor,Nfloors,New
0,1338305,2,9100000,56.4,35.1,7.0,8.942096,15,1,1,1,0,2,5,0
1,1535194,2,52000000,92.0,42.0,18.0,5.830754,1,1,1,0,0,2,30,0
2,1711530,2,6600000,44.5,28.0,6.0,13.741006,13,1,0,1,0,7,9,0
3,1838610,3,6600000,52.3,36.1,6.0,16.486721,10,0,1,1,0,1,5,0
4,1986554,3,13300000,80.4,54.8,8.6,7.735389,10,0,1,1,1,7,8,0
5,2222278,2,7900000,56.0,30.0,6.0,7.068869,5,1,1,0,0,1,8,0
6,2292702,3,15000000,73.5,45.0,10.0,13.470486,9,1,0,0,1,13,17,0
7,2343853,3,9700000,70.4,48.3,7.0,6.834289,12,1,1,1,1,5,6,0
8,2464298,1,14500000,58.0,25.0,15.0,11.137224,12,1,1,0,1,8,18,0
9,2515421,5,20000000,130.0,90.0,8.0,11.272828,11,1,1,1,0,1,5,0


И вот теперь заменяем -999999 на nan:

In [26]:
df_final.replace(-999999, np.nan, inplace=True)
df_final.head(20)

,N,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Tel,Bal,Floor,Nfloors,New
0,1338305,2,9100000,56.4,35.1,7.0,8.942096,15.0,1,1,1,0,2,5,0
1,1535194,2,52000000,92.0,42.0,18.0,5.830754,1.0,1,1,0,0,2,30,0
2,1711530,2,6600000,44.5,28.0,6.0,13.741006,13.0,1,0,1,0,7,9,0
3,1838610,3,6600000,52.3,36.1,6.0,16.486721,10.0,0,1,1,0,1,5,0
4,1986554,3,13300000,80.4,54.8,8.6,7.735389,10.0,0,1,1,1,7,8,0
5,2222278,2,7900000,56.0,30.0,6.0,7.068869,5.0,1,1,0,0,1,8,0
6,2292702,3,15000000,73.5,45.0,10.0,13.470486,9.0,1,0,0,1,13,17,0
7,2343853,3,9700000,70.4,48.3,7.0,6.834289,12.0,1,1,1,1,5,6,0
8,2464298,1,14500000,58.0,25.0,15.0,11.137224,12.0,1,1,0,1,8,18,0
9,2515421,5,20000000,130.0,90.0,8.0,11.272828,11.0,1,1,1,0,1,5,0


Ответ готов!

In [28]:
df_final.to_csv('vao_flats.csv')

Несмотря на то, что кривой интернет не позволил получить желаемую выборку из всех квартир в ВАО, хотя бы половина информации сохранилась. Я во всем разобрался, и в случае необходимости планирую добить ВАО и распарсить еще пару округов (а может, и не пару).
upd: скрипт пробежит еще раз, ибо, видимо, кривой комп еще и где-то налажал и дал перегруз в памяти, и ноутбук перезапустился) так что ответа на самом деле нет :) Но он появится, этот код верный, обработал 4к квартир к моменту апокалипсиса...